In [2]:
import numpy as np
import numpy.linalg as la

# Q6a

In [153]:
import numpy as np
import matplotlib.pyplot as plt
import sys

d = np.load("face_emotion_data.npz")
X = d['X']
y = d['y']

n, p = np.shape(X)

# error rate for regularized least squares
error_RLS = np.zeros((8, 7))
# error rate for truncated SVD
error_SVD = np.zeros((8, 7))

# SVD parameters to test
k_vals = np.arange(9) + 1
param_err_SVD = np.zeros(len(k_vals))

def get_pseudo_inverse_weight(k, X, y):
    U, S, Vt = la.svd(X,full_matrices=False)
    S_p = np.zeros((k, k))
    for i in range(k):
        S_p[i][i] = 1 /S[i]
    psuedo_inverse = Vt.T[:, :k]@(S_p@U[:, :k].T)
    return np.dot(psuedo_inverse, y)

def get_prediction(X, w):
    y_raw = np.dot(X, w)
    y = np.where(y_raw >=0, 1, -1)
    return y

subset_size = n // 8

def get_error_SVD(X, y):
    for i in range(8):
        test_indices = np.arange(i * subset_size, (i+1) * subset_size) % n
        remaining_indices = np.setdiff1d(np.arange(n), test_indices)
        
        for j in range(7):
            val_indices = remaining_indices[j * subset_size:(j+1) * subset_size] % n
            train_indices = np.setdiff1d(remaining_indices, val_indices)
            
            X_train, y_train = X[train_indices], y[train_indices]
            X_val, y_val = X[val_indices], y[val_indices]
            X_test, y_test = X[test_indices], y[test_indices]
            
            best_k = None
            best_error = np.inf
            for k in k_vals:
                w_k = get_pseudo_inverse_weight(k, X_train, y_train)
                y_pred = get_prediction(X_val, w_k)
                error = np.mean(y_pred != y_val)
                if error < best_error:
                    best_k = k
                    best_error = error
            
            w_best_k = get_pseudo_inverse_weight(best_k, X_train, y_train)
            y_pred_test = get_prediction(X_test, w_best_k)
            test_error = np.mean(y_pred_test != y_test)
            
            error_SVD[i, j] = test_error

get_error_SVD(X, y)
mean_err_svd = np.mean(error_SVD.flatten())
print(f"Final error rate for Truncaded SVD: {mean_err_svd}")

# RLS parameters to test
lambda_vals = np.array([0, 0.5, 1, 2, 4, 8, 16])
param_err_RLS = np.zeros(len(lambda_vals))

def get_reg_ls_weight(X, y, lambda_val, k=9):
    U, S, Vt = la.svd(X,full_matrices=False)
    S_p = np.zeros((k, k))
    for i in range(k):
        S_p[i][i] = S[i] /(S[i]**2 + lambda_val)
    w_lambda = Vt.T[:, :k]@(S_p@U[:, :k].T)
    return np.dot(w_lambda, y)

def get_error_RLS(X, y):
    for i in range(8):
        test_indices = np.arange(i * subset_size, (i+1) * subset_size) % n
        remaining_indices = np.setdiff1d(np.arange(n), test_indices)
        
        for j in range(7):
            val_indices = remaining_indices[j * subset_size:(j+1) * subset_size] % n
            train_indices = np.setdiff1d(remaining_indices, val_indices)
            
            X_train, y_train = X[train_indices], y[train_indices]
            X_val, y_val = X[val_indices], y[val_indices]
            X_test, y_test = X[test_indices], y[test_indices]
            
            best_k = None
            best_error = np.inf
            for lambda_val in lambda_vals:
                w_k = get_reg_ls_weight(X_train, y_train, lambda_val, X_train.shape[1])
                y_pred = get_prediction(X_val, w_k)
                error = np.mean(y_pred != y_val)
                if error < best_error:
                    best_k = lambda_val
                    best_error = error
            
            w_best_k = get_reg_ls_weight(X_train, y_train, best_k)
            y_pred_test = get_prediction(X_test, w_best_k)
            test_error = np.mean(y_pred_test != y_test)
            
            error_RLS[i, j] = test_error

get_error_RLS(X, y)
mean_err_rls = np.mean(error_RLS.flatten())
print(f"Final error rate for Regularized LS: {mean_err_rls}")

'''
Now for augmented X:
'''

X_rand = X @ np.random.rand(9, 3)
X = np.hstack((X, X_rand))
    
get_error_SVD(X, y)
mean_err_svd = np.mean(error_SVD.flatten())
print(f"Final error rate for AUGMENTED X Truncaded SVD: {mean_err_svd}")

get_error_RLS(X, y)
mean_err_rls = np.mean(error_RLS.flatten())
print(f"Final error rate for AUGMENTED X Regularized LS: {mean_err_rls}")
    



Final error rate for Truncaded SVD: 0.11160714285714286
Final error rate for Regularized LS: 0.04799107142857143
Final error rate for AUGMENTED X Truncaded SVD: 0.10267857142857142
Final error rate for AUGMENTED X Regularized LS: 0.046875
